# Multi-Agent

임성열 Ph.D. 2025. 02.05, SKALA 수업 학습목적으로만 활용 바랍니다.

Reference: https://devocean.sk.com/blog/techBoardDetail.do?ID=166150&boardType=techBlog

# 환경설정

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install openai==1.55.3 httpx==0.27.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Warning control
# TODO Why..?
import warnings
warnings.filterwarnings('ignore')

In [4]:
from crewai import Agent, Task, Crew

In [5]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'


# What is CrewAI?

Agent(에이전트) 생성하기

Agent를 정의하고, role(역할), goal(목표), backstory(배경 설명)를 제공합니다.

LLM(대규모 언어 모델)은 롤플레잉을 할 때 더 나은 성능을 보이는 것으로 확인되었습니다.

### Agent: Brander

In [6]:
brander = Agent(
role="팀 브랜더",
goal ="""생성형 AI서비스 프로젝트 수행팀에 대해 대외적으로
마케팅하여 팀원을 모집할 수 있게 브랜딩""",
backstory="당신은 생성형AI서비스 프로젝트 수행팀을"
"대외적으로 홍보하고 브랜딩하는 것이 목적임",
allow_delegation=False,
    verbose=True
)

### Agent: Manager

In [7]:
manager = Agent(
role = "팀 매니저",
goal = "팀이 생성형AI서비스 서빙을 성공적으로 수행할 수 있게 관리, 감독 및 지원",
backstory ="단기간내에 역량을 끌어올려야함",
descreption = "일반적인 기업의 복지 및 교육과정을 우선순위화",
allow_delegation=False,
    verbose=True
)

### Agent: Leader

In [8]:
leader = Agent(
    role="팀장",
    goal="팀을 이끌며 목표를 수립하고, 일정 및 팀원들을 관리해 목표를 달성하기 위해 노력함",
    backstory="""
            팀원들의 능력에 대한 파악이 빠르며, 정해진 목표를 이루기 위한 구체적인 일정을 세움.
            불화가 생기는 것을 원치 않으며, 이를 발빠르게 잡아내고 특히 협력을 강조하는 사람임.
            """,
    allow_delegation=False,
    verbose=True
)

### Agent: Follower

In [9]:
follower = Agent(
    role="팀원",
    goal="팀장이 정한 목표에 부함한 의견을 제시하고, 이에 적극적으로 주어진 일을 수행함",
    backstory="""
            성장에 대한 갈망이 있으며, 자신의 능력을 극대화시키고자하는 열망이 있음.
            백엔드, 프론트엔드, UI/UX 등의 기술적 역량을 갖추고 있음.
            """,
    allow_delegation=False,
    verbose=True
)

## Task(작업) 생성하기

- Task를 정의하고, `description`(설명), `expected_output`(예상 결과물), `agent`(수행 에이전트)를 제공합니다.
- Task에 Agent가 parameter로 지정됩니다. 
- Agent의 background보다 task의 description를 더 자세하게 써야 양질의 결과를 얻을 수 있습니다.

### Task: Branding

In [15]:
branding = Task(
    description= "생성형AI 서비스에 대한 최신 트렌드, 주요인물 주목할만한 뉴스를 우선순위화하",
    expected_output="""조이름, 대외적으로 함양하는 가치, 우리팀에 합류함으로써 얻을 수 있는 발전가능성 제시""",
    agent=brander,
)

### Task: Assess

In [16]:
assess = Task(
    description=(
        "일반적인 기업의 복지 및 교육과정을 우선순위화"
    ),
    expected_output= "지원할 수 있는 교육, 지원할 수 있는 편의성",
    agent=manager,
)

### Task: Goal Establishment

In [17]:
goal_establish = Task(
    description=(""""
                팀원의 자기소개를 바탕으로 AI 서비스 개발 프로젝트에 부함하는 프로젝트를 선정하고 이에 대한 목표를 제시한다.
                """),
    expected_output="""
                팀을 하나로 만들기 위한 자기소개를 바탕으로 한 팀명과 Catch Prase를 작성한다.
                이를 바탕으로 팀이 어떤 활동을 해나가면 좋을지를 제시해달라. 
                """,
    agent=leader
)

### Task: Introduce

In [18]:
introduce = Task(
    description=("""
                자신에 대해서 소개하여 이 프로젝트에 참여하게 된 동기를 밝힘. 
                이를 바탕으로 팀이 공통의 목표를 수립하면서도 개인의 목표를 이룰 수 있도록 얘기함.
                """),
    expected_output="자신의 소개에 대해서 설명에 맞는 요소 3가지를 설정하고 이에 대해서 설명함."
                    "이때 팀원은 총 5명임으로, 5개를 작성한다.",
    agent=follower
)

## Crew(팀) 생성하기

- Agent들로 구성된 팀을 생성합니다
- 해당 Agent들이 수행할 작업들을 전달합니다.
    - **참고**: *이 간단한 예시에서는* 작업들이 순차적으로 수행됩니다(즉, 서로 의존적임). 따라서 목록에서의 **작업순서**가 중요합니다.
- `verbose=2`를 설정하면 실행의 모든 로그를 확인할 수 있습니다.


이 설정에서:
1. `planner`가 먼저 콘텐츠를 기획합니다
2. `writer`가 기획된 내용을 바탕으로 글을 작성합니다
3. `editor`가 최종적으로 작성된 글을 검토하고 편집합니다

In [26]:
crew = Crew(
    agents=[follower, leader, manager, brander],
    tasks=[introduce, goal_establish, assess, branding],
    verbose=2
)

2025-02-05 16:32:48,014 - 140225082036224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

In [27]:
# result = crew.kickoff(inputs={"topic": "생성형 AI를 이용한 제약회사 경쟁력 제고 방안"})
result = crew.kickoff()

 [DEBUG]: == Working Agent: 팀원
 [INFO]: == Starting Task: 
                자신에 대해서 소개하여 이 프로젝트에 참여하게 된 동기를 밝힘. 
                이를 바탕으로 팀이 공통의 목표를 수립하면서도 개인의 목표를 이룰 수 있도록 얘기함.
                


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: 

안녕하세요, 저는 팀의 일원으로서 성장에 대한 강한 열망을 가지고 있으며, 제 능력을 최대한 발휘하고자 하는 열정을 가지고 있습니다. 백엔드, 프론트엔드, 그리고 UI/UX에 대한 기술적 역량을 갖추고 있어 다양한 측면에서 팀에 기여할 수 있습니다. 

이번 프로젝트에 참여하게 된 동기는 다음과 같습니다:

1. **성장과 학습의 기회**: 이 프로젝트는 저에게 새로운 기술을 배우고 기존의 기술을 더욱 심화시킬 수 있는 좋은 기회를 제공합니다. 팀의 목표에 부합하는 솔루션을 제시하고 이를 실행함으로써, 개인적으로도 성장할 수 있는 발판을 마련하고자 합니다.

2. **팀워크와 협업**: 다섯 명의 팀원과 함께 협력하여 프로젝트를 수행함으로써, 저는 다양한 관점과 아이디어를 접할 수 있습니다. 이는 저의 사고의 폭을 넓히고, 더 나아가 팀워크를 통해 더 나은 결과물을 만들어낼 수 있는 기회를 제공합니다.

3. **결과 지향적인 목표 달성**: 팀장이 설정한 목표를 충실히 이행하면서, 저의 개인 목표도 달성할 수 있도록 최선을 다할 것입니다. 이는 개인의 성과와 팀의 성공을 동시에 이루고자 하는 저의 열정을 반영합니다.

이러한 동기를 바탕으로, 팀의 공통 목표를 수립하면서 각자의 개인 목표도 함께 달성할 수 있도록 최선을 다하겠습니다. 감사합니다.

> Finished chain.
 [DEBUG]: == [팀원] Task output: 안녕하세요, 저는 팀의 

In [28]:
# Display the results of your execution as markdown in the notebook.
from IPython.display import Markdown
Markdown(result)

"Synergy Innovators" 팀은 생성형 AI 서비스 프로젝트 수행팀으로서, 최신 AI 기술의 선구자가 되고자 하는 열정과 혁신을 가진 인재들을 모집하고 있습니다. 대외적으로 함양하는 가치는 다음과 같습니다:

1. **혁신과 창의성**: 우리는 혁신적인 AI 솔루션을 통해 사용자 중심의 서비스를 제공하여 세상을 변화시키고자 합니다. 창의적인 아이디어를 발굴하고 이를 실현할 수 있는 환경을 조성합니다.

2. **협력과 성장**: 팀 내의 협력을 통해 시너지를 창출하고, 팀원들이 개인적으로도 성장할 수 있는 기회를 제공합니다. 다양한 배경의 팀원들이 서로의 지식과 경험을 공유하여 함께 발전합니다.

3. **지속적인 학습과 개발**: 최신 트렌드와 기술에 대한 지속적인 학습을 통해 팀의 역량을 강화하고, 이를 통해 시장에서의 경쟁력을 확보합니다.

"Synergy Innovators" 팀에 합류함으로써 얻을 수 있는 발전 가능성은 무궁무진합니다. 팀원들은 최신 AI 기술을 접하고, 이를 실무에 적용하면서 기술적 역량을 극대화할 수 있습니다. 또한, 팀 내 멘토링 프로그램과 유연 근무제를 통해 개인의 업무와 학습을 조율하며, 자신만의 커리어 경로를 설계할 수 있습니다. 다양한 복지 혜택과 정신적 건강 지원은 팀원들이 최상의 상태에서 업무에 몰입할 수 있도록 돕습니다.

이러한 가치와 기회를 통해 우리는 생성형 AI 서비스 분야에서의 리더로서 자리매김하고자 합니다. 여러분의 참여가 "Synergy Innovators" 팀의 성공에 큰 기여를 할 것입니다. 함께 혁신을 이끌어 갈 인재를 기다리겠습니다.